# Medical Ministrations through Web Scraping

## PRESENTED BY:

## Niketha Sabesan - 19MIC0035 
## Nivethitha - 19MIC0030
## J.N Shreyah - 19MIC0009 
## Pranauv A J - 19MIC0016 
## Shyam R - 19MIC0017

In [98]:
# import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')

In [2]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split,cross_val_score
import math
import operator
import pickle
import re
from nltk.stem import WordNetLemmatizer
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from statistics import mean
from nltk.corpus import wordnet 
import requests
from bs4 import BeautifulSoup
from itertools import combinations
from time import time
from collections import Counter
import operator
import warnings
from Treatment import diseaseDetail
warnings.simplefilter("ignore")

In [3]:
df=pd.read_csv("Dataset/dis_dataset.csv")
documentname_list=list(df['label_dis'])
df=df.iloc[:,1:]
columns_name=list(df.columns)
documentname_list=list(documentname_list)

N=len(df)
M=len(columns_name)

idf={}
for col in columns_name:
  temp=np.count_nonzero(df[col])
  idf[col]=np.log(N/temp)
    
tf={}
for i in range(N):
  for col in columns_name:
    key=(documentname_list[i],col)
    tf[key]=df.loc[i,col]

    
tf_idf={}
for i in range(N):
  for col in columns_name:
    key=(documentname_list[i],col)
    tf_idf[key]=float(idf[col])*float(tf[key])

D = np.zeros((N, M),dtype='float32')
for i in tf_idf:
    sym = columns_name.index(i[1])
    dis=documentname_list.index(i[0])
    D[dis][sym] = tf_idf[i]

def cosine_dot(a, b):
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0
    else:
        temp = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
        return temp


def convert_tolowercase(data):
    return data.lower()


def regextokenizer_func(data):
    tokenizer = RegexpTokenizer(r'\w+')
    data = tokenizer.tokenize(data)
    return data


def gen_vector(tokens):
    Q = np.zeros(M)
    counter = Counter(tokens)
    query_weights = {}
    for token in np.unique(tokens):
        tf = counter[token]
        try:
          idf_temp=idf[token]
        except:
          pass
        try:
            ind = columns_name.index(token)
            Q[ind] = tf*idf_temp
        except:
            pass
    return Q


def tf_idf_score(k, query):
    query_weights = {}
    for key in tf_idf:
        if key[1] in query:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
  
    l = []
    for i in query_weights[:k]:
        l.append(i)
    return l

 
def cosine_similarity(k, query):
    d_cosines = []
    query_vector = gen_vector(query)
    for d in D:
        d_cosines.append(cosine_dot(query_vector, d))
    out = np.array(d_cosines).argsort()[-k:][::-1]
  
    final_display_disease={}
    for lt in set(out):
      final_display_disease[lt] = float(d_cosines[lt])
    return final_display_disease

In [4]:
df

,* crusty red nodules,* deposits of cholesterol on the eye lids,* digestive issues,* fatigue,1–2 cm lumps under the skin,a change in breast shape,a lump in a breast,a newly inverted nipple,a red scaly patch of skin on the breast,a round,...,weakness and fatigue,webbed neck,weight loss,wheeze,wheezing,widespread pain,withdrawal occurring when stopping,yellow skin,yellowish coloration of skin and sclera,yellowish skin
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
187,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
188,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
idf

{'* crusty red nodules': 5.247024072160486,
 '* deposits of cholesterol on the eye lids': 5.247024072160486,
 '* digestive issues': 5.247024072160486,
 '* fatigue': 5.247024072160486,
 '1–2 cm lumps under the skin': 5.247024072160486,
 'a change in breast shape': 5.247024072160486,
 'a lump in a breast': 5.247024072160486,
 'a newly inverted nipple': 5.247024072160486,
 'a red scaly patch of skin on the breast': 5.247024072160486,
 'a round': 5.247024072160486,
 'abdominal bloating': 5.247024072160486,
 'abdominal cramps': 5.247024072160486,
 'abdominal pain': 3.1675825304806504,
 'abnormal bleeding': 5.247024072160486,
 'abnormal facial features': 5.247024072160486,
 'abnormal reflexes': 5.247024072160486,
 'abnormal sensations': 5.247024072160486,
 'abnormally frequent': 5.247024072160486,
 'abnormally small head and brain': 5.247024072160486,
 'abscess': 5.247024072160486,
 'acne': 5.247024072160486,
 'acquiring and drinking alcohol taking up a lot of time': 5.247024072160486,
 'an 

In [6]:
tf

{('Abdominal Pain', '* crusty red nodules'): 0,
 ('Abdominal Pain', '* deposits of cholesterol on the eye lids'): 0,
 ('Abdominal Pain', '* digestive issues'): 0,
 ('Abdominal Pain', '* fatigue'): 0,
 ('Abdominal Pain', '1–2 cm lumps under the skin'): 0,
 ('Abdominal Pain', 'a change in breast shape'): 0,
 ('Abdominal Pain', 'a lump in a breast'): 0,
 ('Abdominal Pain', 'a newly inverted nipple'): 0,
 ('Abdominal Pain', 'a red scaly patch of skin on the breast'): 0,
 ('Abdominal Pain', 'a round'): 0,
 ('Abdominal Pain', 'abdominal bloating'): 0,
 ('Abdominal Pain', 'abdominal cramps'): 0,
 ('Abdominal Pain', 'abdominal pain'): 1,
 ('Abdominal Pain', 'abnormal bleeding'): 0,
 ('Abdominal Pain', 'abnormal facial features'): 0,
 ('Abdominal Pain', 'abnormal reflexes'): 0,
 ('Abdominal Pain', 'abnormal sensations'): 0,
 ('Abdominal Pain', 'abnormally frequent'): 0,
 ('Abdominal Pain', 'abnormally small head and brain'): 0,
 ('Abdominal Pain', 'abscess'): 0,
 ('Abdominal Pain', 'acne'): 0,


In [7]:
tf_idf

{('Abdominal Pain', '* crusty red nodules'): 0.0,
 ('Abdominal Pain', '* deposits of cholesterol on the eye lids'): 0.0,
 ('Abdominal Pain', '* digestive issues'): 0.0,
 ('Abdominal Pain', '* fatigue'): 0.0,
 ('Abdominal Pain', '1–2 cm lumps under the skin'): 0.0,
 ('Abdominal Pain', 'a change in breast shape'): 0.0,
 ('Abdominal Pain', 'a lump in a breast'): 0.0,
 ('Abdominal Pain', 'a newly inverted nipple'): 0.0,
 ('Abdominal Pain', 'a red scaly patch of skin on the breast'): 0.0,
 ('Abdominal Pain', 'a round'): 0.0,
 ('Abdominal Pain', 'abdominal bloating'): 0.0,
 ('Abdominal Pain', 'abdominal cramps'): 0.0,
 ('Abdominal Pain', 'abdominal pain'): 3.1675825304806504,
 ('Abdominal Pain', 'abnormal bleeding'): 0.0,
 ('Abdominal Pain', 'abnormal facial features'): 0.0,
 ('Abdominal Pain', 'abnormal reflexes'): 0.0,
 ('Abdominal Pain', 'abnormal sensations'): 0.0,
 ('Abdominal Pain', 'abnormally frequent'): 0.0,
 ('Abdominal Pain', 'abnormally small head and brain'): 0.0,
 ('Abdominal P

In [16]:
def synonyms(term):
    synonyms = []
    response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.content,  "html.parser")
    try:
        container=soup.find('section', {'class': 'MainContentContainer'}) 
        row=container.find('div',{'class':'css-ixatld e15rdun50'})
        row = row.find_all('li')
        for x in row:
            synonyms.append(x.get_text())
    except:
        None
    for syn in wordnet.synsets(term):
        synonyms+=syn.lemma_names()
    return set(synonyms)
    

In [12]:
splitter = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [13]:
df_norm = pd.read_csv("Dataset/dis_dataset.csv") 
Y = df_norm.iloc[:, 0:1]
X = df_norm.iloc[:, 1:]
dataset_symptoms = list(X.columns)
diseases = list(set(Y['label_dis']))
diseases.sort()

# Symptoms Entering 

In [14]:
print("Hello! Welcome to our Virtual-Chatbot")
user_symptoms = str(input("\nEnter symptoms : (comma seperated)\n")).lower().split(',')
processed_user_symptoms=[]
for sym in user_symptoms:
    sym=sym.strip()
    sym=sym.replace('-',' ')
    sym=sym.replace("'",'')
    sym = ' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym)])
    processed_user_symptoms.append(sym)  

Hello! Welcome to our Virtual-Chatbot

Enter symptoms : (comma seperated)
heart break 


In [17]:
user_symptoms = []
for user_sym in processed_user_symptoms:
    user_sym = user_sym.split()
    str_sym = set()
    for comb in range(1, len(user_sym)+1):
        for subset in combinations(user_sym, comb):
            subset=' '.join(subset)
            subset = synonyms(subset) 
            str_sym.update(subset)
    str_sym.add(' '.join(user_sym))
    user_symptoms.append(' '.join(str_sym).replace('_',' '))
print("Synonyms of the entered symptoms\n")
print(user_symptoms)

Synonyms of the entered symptoms

['sensitivity  prisonbreak get around sorrow  love  regret  ruin fault severance sympathy  discontinue time out prison-breaking wear out centre split  nitty-gritty relegate rift schism  temperament  go erupt spunk woe  inwardness break out tenderness  founder break-dance understanding  break in develop expose rift  breakage infract burst break up separate split crack conk out give way gist recrudesce heartache  pain  geological fault die response  collapse gash  gap  bitterness  break away give away zest  damp affectionateness check palate  meat gaolbreak soul  nerve nub split up care  ticker let out humanity  philia intermission kick downstairs agony  division  tenderness affection grief  bring out shift sentiment  broken heart  disclose bosom cleft  go against gap disposition  fall in break off wear pump torture  let on falling out bust anguish  faulting remorse  bump fondness inclination  concern  nature  substance bale  get out fracture  snap off h

In [18]:
found_symptoms = set()
for idx, data_sym in enumerate(dataset_symptoms):
    data_sym_split=data_sym.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1
                found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)

In [19]:
print("Matching symptoms from the data entered")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)

select_list = input("\n Hello User! Select more relevant symptoms (space seperated) \n").split()

dis_list = set()
final_symp = [] 
counter_list = []
for idx in select_list:
    symp=found_symptoms[int(idx)]
    final_symp.append(symp)
    dis_list.update(set(df_norm[df_norm[symp]==1]['label_dis']))
   
for dis in dis_list:
    row = df_norm.loc[df_norm['label_dis'] == dis].values.tolist()
    row[0].pop(0)
    for idx,val in enumerate(row[0]):
        if val!=0 and dataset_symptoms[idx] not in final_symp:
            counter_list.append(dataset_symptoms[idx])
             
dict_symp = dict(Counter(counter_list))
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)   


Matching symptoms from the data entered
0 : red purple on darker skin
1 : pain around the ear
2 : joint pain
3 : birth of a baby at younger than 37 weeks gestational age
4 : and loss of appetite
5 : a lump in a breast
6 : coughing up mucus
7 : acquiring and drinking alcohol taking up a lot of time
8 : reddish eye
9 : vomiting or diarrhea and muscle aches rash
10 : sensitivity to light
11 : loss of smell
12 : feeling tired
13 : hard swelling of the skin
14 : change in taste
15 : pelvic pain
16 : episodes of severe
17 : opening in the upper lip that may extend into the nose or palate
18 : or stomach pain
19 : * deposits of cholesterol on the eye lids
20 : patches of white skin
21 : difficulty cutting down
22 : sudden loss of muscle strength
23 : usually on the scalp
24 : neck pain
25 : but can cause spots in the eye and vision loss
26 : and the presence of repetitive behavior or restricted interests
27 : lumps/bumps in the neck
28 : persistent rough white or red patch in the mouth lastin

In [ ]:
found_symptoms=[]
count=0
for tup in dict_symp_tup:
    count+=1
    found_symptoms.append(tup[0])
    #print(tup)
    if count%5==0 or count==len(dict_symp_tup):
        print("\nCooccurring symptoms:")
        for idx,ele in enumerate(found_symptoms):
            print(idx,":",ele)
        select_list = input("Do you have any of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:\n").lower().split();
        if select_list[0]=='no':
            break
        if select_list[0]=='-1':
            found_symptoms = [] 
            continue
        for idx in select_list:
            final_symp.append(found_symptoms[int(idx)])
        found_symptoms = []    


Cooccurring symptoms:
0 : headache
1 : malaise
2 : muscle pain
3 : parotitis and non specific symptoms such as fever


In [15]:
print("Final list of Symptoms used for prediction are : \n")
for val in final_symp:
    print(val)

Final list of Symptoms used for prediction are : 

headache
muscle pain
cough
sore throat


In [16]:
k = 10

topk1=tf_idf_score(k,final_symp)
print(f"\nTop {k} diseases predicted based on TF_IDF Matching :\n")
i = 0
topk1_index_mapping = {}
for key, score in topk1:
  print(f"{i}. Disease : {key} \t Score : {round(score, 2)}")
  topk1_index_mapping[i] = key
  i += 1

select = input("\nTo enter more details about the disease? Enter index of disease or '-1' to discontinue:\n")
if select!='-1':
    dis=topk1_index_mapping[int(select)]
    print()
    print(diseaseDetail(dis))


Top 10 diseases predicted based on TF_IDF Matching :

0. Disease : Pharyngoconjunctival Fever 	 Score : 8.6
1. Disease : Ebola Virus Disease (EVD) 	 Score : 5.55
2. Disease : Crimean Congo haemorrhagic fever (CCHF) 	 Score : 5.37
3. Disease : Leptospirosis 	 Score : 5.37
4. Disease : Monkeypox 	 Score : 5.37
5. Disease : Mumps 	 Score : 5.37
6. Disease : Severe Acute Respiratory Syndrome (SARS) 	 Score : 5.37
7. Disease : Yellow Fever 	 Score : 5.37
8. Disease : Mucormycosis 	 Score : 4.96
9. Disease : Nipah virus infection 	 Score : 4.96

To enter more details about the disease? Enter index of disease or '-1' to discontinue:
4

Monkeypox
Other names -  Monkeypox 
Specialty -  Infectious disease   
Symptoms -  Fever, headache, muscle pains, shivering, blistering rash, swollen lymph nodes   
Complications -  Secondary infections, eye infection, visual loss, scarring,   
Usual onset -  5–21 days post exposure   
Duration -  2 to 4 weeks   
Types -  Central African (Congo Basin), West Af

In [17]:
topk2=cosine_similarity(k,final_symp)
print(f"Top {k} disease based on Cosine Similarity Matching :\n ")
topk2_sorted = dict(sorted(topk2.items(), key=lambda kv: kv[1], reverse=True))
j = 0
topk2_index_mapping = {}
for key in topk2_sorted:
  print(f"{j}. Disease : {diseases[key]} \t Score : {round(topk2_sorted[key], 2)}")
  topk2_index_mapping[j] = diseases[key]
  j += 1

    
select = input("\nMore details about the disease? Enter index of disease or '-1' to discontinue and close the system:\n")
if select!='-1':
    dis=topk2_index_mapping[int(select)]
    print()
    print(diseaseDetail(dis))

Top 10 disease based on Cosine Similarity Matching :
 
0. Disease : Leptospirosis 	 Score : 0.6
1. Disease : Poliomyelitis 	 Score : 0.55
2. Disease : Middle East respiratory syndrome coronavirus (MERS‐CoV) 	 Score : 0.35
3. Disease : Yellow Fever 	 Score : 0.34
4. Disease : Nipah virus infection 	 Score : 0.33
5. Disease : Diphtheria 	 Score : 0.33
6. Disease : Crimean Congo haemorrhagic fever (CCHF) 	 Score : 0.31
7. Disease : Severe Acute Respiratory Syndrome (SARS) 	 Score : 0.3
8. Disease : Rubella 	 Score : 0.29
9. Disease : Cough 	 Score : 0.28

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
0

Leptospirosis
Other names -  Rat fever,   
Specialty -  Infectious disease 
Symptoms -  None, headaches, muscle pains, fevers   
Complications -  Bleeding from the lungs, meningitis, kidney failure   
Usual onset -  One to two weeks   
Causes -   Leptospira  typically spread by rodents   
Risk factors -  Exposure to infected animals, es